## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-01-10-58 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/11/us/cha...
1,2025-09-12-01-10-28 +0000,bbc,Ryan Routh cut off by judge as trial over atte...,https://www.bbc.com/news/articles/c8d793z4mzgo...
2,2025-09-12-01-10-01 +0000,nyt,Nepal’s Social Media Ban Backfires as Politics...,https://www.nytimes.com/2025/09/11/world/asia/...
3,2025-09-12-01-09-11 +0000,nyt,He was the last person to question Charlie Kirk.,https://www.nytimes.com/live/2025/09/11/us/cha...
4,2025-09-12-01-06-29 +0000,wapo,Live updates: Manhunt continues for Charlie Ki...,https://www.washingtonpost.com/nation/2025/09/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,kirk,72
25,charlie,63
9,trump,42
57,shooting,17
52,killing,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
51,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,239
301,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,219
38,2025-09-11-23-30-12 +0000,nypost,"Bluesky posters threaten Trump, Elon Musk, Ben...",https://nypost.com/2025/09/11/us-news/bluesky-...,210
261,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free-sp...,https://www.latimes.com/california/story/2025-...,208
161,2025-09-11-18-02-59 +0000,nyt,Trump Says He Will Award Charlie Kirk the Pres...,https://www.nytimes.com/2025/09/11/us/charlie-...,205


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
51,239,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
301,58,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
242,49,2025-09-11-13-50-47 +0000,latimes,FBI releases photos of 'person of interest' in...,https://www.latimes.com/world-nation/story/202...
165,49,2025-09-11-17-58-01 +0000,latimes,New Paramount owner already has his sights on ...,https://www.latimes.com/entertainment-arts/bus...
118,47,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
39,42,2025-09-11-23-24-07 +0000,nypost,Hospital bigwig finally jailed for vicious att...,https://nypost.com/2025/09/11/us-news/hospital...
305,42,2025-09-11-08-27-23 +0000,nypost,JD Vance pens heartfelt tribute to Charlie Kir...,https://nypost.com/2025/09/11/us-news/jd-vance...
282,42,2025-09-11-10-00-00 +0000,latimes,California legislators strike last-minute deal...,https://www.latimes.com/california/story/2025-...
277,41,2025-09-11-10-32-06 +0000,nypost,UK PM Starmer fires ambassador to US ahead of ...,https://nypost.com/2025/09/11/world-news/uks-s...
185,34,2025-09-11-17-00-00 +0000,wsj,Economists worry that a ‘universal basic incom...,https://www.wsj.com/economy/a-guaranteed-incom...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
